In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/25740/head accelerate


  Cloning https://github.com/huggingface/transformers.git (to revision refs/pull/25740/head) to /tmp/pip-req-build-b3iahw0d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-b3iahw0d
  Running command git fetch -q https://github.com/huggingface/transformers.git refs/pull/25740/head
  Running command git checkout -q c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Resolved https://github.com/huggingface/transformers.git to commit c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00
 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/MultilingualLLMBias"

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

# Enable these models to reproduce results

model = "codellama/CodeLlama-7b-Instruct-hf"
# model = "codellama/CodeLlama-7b-Python-hf" #"codellama/CodeLlama-7b-hf"
# model = "codellama/CodeLlama-13b-Python-hf"
# model = "codellama/CodeLlama-13b-Instruct-hf"

dirname = model.split("./")[-1]
result_dir = path+f"results-direct-translation-{dirname}"
!mkdir -p $result_dir

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    tokenizer = tokenizer
)


(…)ct-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)-Instruct-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

(…)13b-Instruct-hf/resolve/main/config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
import pandas
import tqdm
languages = ["hi","ru","es","ja","zh-cn"]
language_map = {"hi":"Hindi", "ru":"Russian", "es":"Spanish", "ja":"Japanese","zh-cn":"Chinese"}

def create_programing_template(question):
  system = "Provide answers in Python"
  user = question
  prompt = f"<s>[INST] <<SYS>>\\n{system}\\n<</SYS>>\\n\\n{user}[/INST]"
  return prompt

def filter_translations(res):
  # We intend to extract only English sentences and not tags
  for r in res.split("\n"):
    if "write" in r.lower():
      return r.strip()
    elif "sentence" in r.lower() and "translate" not in r.lower():
      return r.strip()
    elif "function" in r.lower():
      return r.strip()
  # sometimes return the original text, if translation is not found
  for r in res.split("\n"):
    if len(r)>15 and "translate" not in r.lower():
      return r.strip()


for language in tqdm.tqdm(languages):

  def create_translation_template(question):
    map = language_map[language]
    system = f"Translate the following sentence from {map} to English"
    user = question
    prompt = f"<s><<SYS>>\n{system}\n<</SYS>>\n\n{user}"
    return prompt


  df = pandas.read_csv(f"/content/drive/MyDrive/MultilingualLLMBias/test.{language}.sanitized.csv")
  all_data = []

  translation_prompts = df["prompt"].apply(create_translation_template)
  df["translation_prompt"] = translation_prompts
  tr_prompts = translation_prompts.to_list()

  pipeline.tokenizer.pad_token_id = pipeline.model.config.eos_token_id

  sequences = pipeline(
      tr_prompts,
      do_sample=True,
      top_k=10,
      temperature=0.1,
      top_p=0.95,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=500,
      add_special_tokens=False,
      batch_size = 16,
  )

  for sequence in sequences:
    res = sequence[0]['generated_text']
    final_tr = filter_translations(res)
    all_data.append(final_tr)
  df["en_translation"] = all_data

  all_prog_data = []

  prompts_df = df["en_translation"].apply(create_programing_template)
  df["prompt_modified"] = prompts_df
  prompts = prompts_df.to_list()

  sequences = pipeline(
      prompts,
      do_sample=True,
      top_k=10,
      temperature=0.1,
      top_p=0.95,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=200,
      add_special_tokens=False,
      batch_size = 16,
  )

  for sequence in sequences:
    all_prog_data.append(sequence[0]['generated_text'])
  df["results"] = all_prog_data
  df.to_csv(f"{result_dir}/test.{language}.sanitized.translated.results.csv")

  0%|          | 0/5 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pa